## 1 Install library（ライブラリをインストール）

In [ ]:
!pip -q install smolagents
!pip -q install litellm

In [ ]:
import os
from google.colab import userdata
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
os.environ["GEMINI_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
import pandas as pd
df = pd.read_csv('bank-additional-full.csv', sep=';')
df

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


## 2 Make prompt（プロンプトの作成）

In [ ]:
prompt = "you are an awesome data analyst.Your task is to answer the question based on data analysis.\
          Please follow these instructions:\
          1. Review the data imported to ensure every feature is accounted for.\
          2. Data should be analysed into a logical, step-by-step order, using the specified prosesure and library.\
          3. You should import data from the directory '/content', do not use web search.\
          4. You should use pandas to extract data.\
          5. You should use HistGradientBoostingClassifier in sklearn to analyse data.\
          6. You should develop the predictive model to analyse data.\
          7. You should present the results reqired by users in tabular format.\
          8. Use the following format\
              - **Main actions are numbered** (e.g., 1, 2, 3).\
              - **result of each action shoud be presented in tabular format** \
          9. Results should be saved in 'output.csv' file.\
          "

## 3 Create agent with gemini-2.0-flash-thinking-exp-01-21（gemini-2.0-flash-thinking-exp-01-21を使いエージェント作成）

In [ ]:
from smolagents import CodeAgent, HfApiModel, LiteLLMModel

model = LiteLLMModel(model_id="gemini/gemini-2.0-flash-thinking-exp-01-21")
agent = CodeAgent(tools=[], model=model, add_base_tools=True, \
                  additional_authorized_imports=['pandas', 'math', 'queue', 'random', \
                 'time', 'sklearn', "matplotlib.pyplot"," seaborn", 'shap'], \
                  max_steps=12)


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


##  4 Predict probability of purchase（購入確率の予測）

In [ ]:
%%time

question= " Could you provide the probability of purchase for each customer by sklearn? Please follow this instruction.\
          1. Import file of 'bank-additional-full.csv' from in the directory '/content', do not use web search.\
          2. You should tell the 10 customer ID who has the highest probability of purchase. \
          3. AUC metric should be provided to evaluate the model. \
          4. Devide 5 bins and provide the distrubution of customer according to the probability of purchase ? \
          "

output=agent.run(
    prompt+question,
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ you are an awesome data analyst.Your task is to answer the question based on data analysis.          Please     │
│ follow these instructions:          1. Review the data imported to ensure every feature is accounted for.       │
│ 2. Data should be analysed into a logical, step-by-step order, using the specified prosesure and library.       │
│ 3. You should import data from the directory '/content/drive/MyDrive/01_Agent', do not use web search.          │
│ 4. You should use pandas to extract data.          5. You should use HistGradientBoostingClassifier in sklearn  │
│ to analyse data.          6. You should develop the predictive model to analyse data.          7. You should    │
│ present the results reqired by users in tabular format.          8. Use the following format              -     │
│ **Main actions are numbered** (e.g., 1, 2, 3).              - **result of each action shoud be presented in     │
│ tabular format**           9. Results should be saved in 'output.csv' file.           Could you provide the     │
│ probability of purchase for each customer by sklearn? Please follow this instruction.          1. Import file   │
│ of 'bank-additional-full.csv' from in the directory '/content/drive/MyDrive/01_Agent', do not use web search.   │
│ 2. You should tell the 10 customer ID who has the highest probability of purchase.           3. AUC metric      │
│ should be provided to evaluate the model.           4. Devide 5 bins and provide the distrubution of customer   │
│ according to the probability of purchase ?                                                                      │
│                                                                                                                 │
╰─ LiteLLMModel - gemini/gemini-2.0-flash-thinking-exp-01-21 ─────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
  from sklearn.model_selection import train_test_split                                                             
  from sklearn.ensemble import HistGradientBoostingClassifier                                                      
  from sklearn.metrics import roc_auc_score                                                                        
  import numpy as np                                                                                               
                                                                                                                   
  file_path = '/content/drive/MyDrive/01_Agent/bank-additional-full.csv'                                           
  df = pd.read_csv(file_path, sep=';')                                                                             
  print(df.head())                                                                                                 
  print(df.shape)                                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Warning to user: Code execution failed due to an unauthorized import - Consider passing said import under 
`additional_authorized_imports` when initializing your CodeAgent.

Code execution failed at line 'import numpy as np' due to: InterpreterError:Import of numpy is not allowed. 
Authorized imports are: ['unicodedata', 'random', 'time', 'stat', 're', 'datetime', 'sklearn', 'queue', 
'collections', 'pandas', 'math', 'statistics', ' seaborn', 'itertools', 'matplotlib.pyplot', 'shap']

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 7 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Results are saved to 'output.csv' and printed below:\n" + results_df.to_string())                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Results are saved to 'output.csv' and printed below:
                                                      Top 10 Customer IDs  AUC Score                    Probability
Bin Distribution
0  [40284, 40556, 24054, 40855, 40759, 24059, 40359, 39654, 40954, 40134]   0.947989  {0: 32899, 1: 2727, 2: 2708, 
3: 1761, 4: 1093}

[Step 7: Duration 0.80 seconds| Input tokens: 36,817 | Output tokens: 1,524]

CPU times: user 4.64 s, sys: 176 ms, total: 4.82 s
Wall time: 45.5 s


In [ ]:
import pandas as pd
df = pd.read_csv('output.csv')
df

,Top 10 Customer IDs,AUC Score,Probability Bin Distribution
0,"[40284, 40556, 24054, 40855, 40759, 24059, 403...",0.947989,"{0: 32899, 1: 2727, 2: 2708, 3: 1761, 4: 1093}"


from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_0.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['AUC Score']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'AUC Score'}, axis=1)
              .sort_values('AUC Score', ascending=True))
  xs = counted['AUC Score']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_1.sort_values('AUC Score', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('AUC Score')
_ = plt.ylabel('count()')